# S2 Geomedian plus TMAD

In [1]:
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from pyproj import Transformer

import datacube

import sys
sys.path.append('../Scripts')
from deafrica_dask import create_local_dask_cluster
from s2_geomedian import run_tile


In [2]:
create_local_dask_cluster(n_workers=14,threads_per_worker=1)

Client Scheduler: tcp://127.0.0.1:45365 Dashboard: /user/fangfy/proxy/8787/status,Cluster Workers: 14 Cores: 14 Memory: 104.37 GB


## Generate seasonal geomedian for 2017 and 2019

In [3]:

def run_gm_tmad(lat, lon, tilesize, cloud_buffer, maxcloud, annual_only=False, fourmonthly=False, outpath='s2_geomedian_tmad', 
                bands=['blue','green','red','nir_2','swir_1']):
    
    # convert to projected centre coordinates
    transformer = Transformer.from_crs("epsg:4326", "epsg:6933", always_xy=True)
    x0, y0 = transformer.transform(lon, lat)

    x = (x0-tilesize/2, x0+tilesize/2)
    y = (y0-tilesize/2, y0+tilesize/2)

    if annual_only:
        times = [("2019-01-01", "2019-12-31")]
    elif fourmonthly:
        times = [
            ("2019-01-01", "2019-04-30"),
            ("2019-05-01", "2019-08-31"),
            ("2019-09-01", "2019-12-31"),]
    else:
        times = [
        # quarter
        ("2019-01-01", "2019-03-31"),
        ("2019-04-01", "2019-06-30"),
        ("2019-07-01", "2019-09-30"),
        ("2019-10-01", "2019-12-31"),
        # 6 month
        #("2019-01-01", "2019-06-30"),
        #("2019-07-01", "2019-12-31"),
        # 12 month
        ("2019-01-01", "2019-12-31"),
        ]

    dask_chunks = {'time': -1, 'x':200, 'y': 200}

    filenames = []
    for time in times:
        if cloud_buffer>0:
            filename = "%s/cloud_cover_0_%d/s2_%s_%s_%d_%d_buffer%d.nc"%(outpath, maxcloud, time[0], time[1], x0, y0, cloud_buffer)
        else:
            filename = "%s/cloud_cover_0_%d/s2_%s_%s_%d_%d.nc"%(outpath, maxcloud, time[0], time[1], x0, y0)
        print(filename)
        filenames.append(filename)
    
        gm = run_tile(x, y, time, filename,
                  cloud_labels=[8,9,10], cloud_buffer=cloud_buffer, bad_labels=[0,1,3], 
                  bands=bands, # only testing a few bands for speed
                  redo=False, tmad=True,
                  dask_chunks = dask_chunks, cloud_cover=[0, maxcloud],
                 )


In [ ]:
maxcloud = 100
tilesize = 20000
cloud_buffer=0

latlons = [(-33.7, 19.515),
           ]

bands=['blue','green','red','nir','swir_1','swir_2']

for lat, lon in latlons:
    run_gm_tmad(lat, lon, tilesize, cloud_buffer, maxcloud, annual_only=True, bands=bands)


## Eastern Africa locations

In [4]:
maxcloud = 100
tilesize = 50000 # 50 km
cloud_buffer=0

latlons = [(-0.629425, 34.135745),
            (11.521710, 37.489409),
            #(-0.295307, 38.442294),
           (12.263660, 37.024471),
           ]

bands=['blue','green','red','red_edge_1','red_edge_2','red_edge_3','nir_1', 'nir_2','swir_1','swir_2']

for lat, lon in latlons:
    run_gm_tmad(lat, lon, tilesize, cloud_buffer, maxcloud, annual_only=True, bands=bands)


s2_geomedian_tmad/cloud_cover_0_100/s2_2019-01-01_2019-12-31_3293631_-80296.nc
s2_geomedian_tmad/cloud_cover_0_100/s2_2019-01-01_2019-12-31_3617213_1460236.nc
s2_geomedian_tmad/cloud_cover_0_100/s2_2019-01-01_2019-12-31_3572353_1552909.nc
Using pixel quality parameters for Sentinel 2
Finding datasets
    s2_l2a
Applying pixel quality/cloud mask
Re-scaling Sentinel-2 data
Returning 73 time steps as a dask array
73 images during 2019-01-01 - 2019-12-31


In [ ]:
import xarray as xr
import numpy as np

fname = "s2_2019-01-01_2019-12-31_1882929_-4061168.nc"
fname = "s2_2019-01-01_2019-12-31_3293631_-80296.nc"
#fname= "s2_2019-01-01_2019-12-31_3617213_1460236.nc"
data = xr.open_dataset(f"s2_geomedian_tmad/cloud_cover_0_100/{fname}")
data['bcmad'] = -np.log(data.bcmad.where(data.bcmad>0))
data['emad'] = -np.log(data.emad.where(data.emad>0))
data['smad'] = -np.log(data.smad.where(data.smad>0))

data[['emad', 'smad', 'bcmad']].to_array().plot.imshow(robust=True, figsize=(10,10));
data[['swir_1', 'nir_1', 'red']].to_array().plot.imshow(robust=True, figsize=(10,10));